# Computerphysik Programmiertutorial 13
Prof. Dr. Matteo Rizzi und Dr. Markus Schmitt - Institut für Theoretische Physik, Universität zu Köln
&nbsp;

**ILIAS**: [https://www.ilias.uni-koeln.de/ilias/goto_uk_crs_3862489.html](https://www.ilias.uni-koeln.de/ilias/goto_uk_crs_3862489.html)

**Github**: [https://github.com/markusschmitt/compphys2021](https://github.com/markusschmitt/compphys2021)

**Inhalt dieses Notebooks**: Cluster finden mit Rekursion, Verkehrssimulation, Herangehensweise für numerische Experimente

# Cluster finden mit Rekursion

Wir wollen in einem zufällig besetzten Quadratgitter den größten zusammenhängenden Cluster finden. Am Ende wollen wir die Clustergröße als Funktion der Besetzungswahrscheinlichkeit untersuchen.

Erzeugen wir also zunächst ein Quadratgitter:

In [ ]:
using PyPlot, Statistics, Random

In [ ]:
function get_configuration(N, p)
    return reshape([rand()<p for j in 1:N*N], (N,N))
end

G=get_configuration(10,0.6)

imshow(G)
axis("off");

Die Aufgabe zusammenhängende Cluster zu finden eignet sich gut für eine rekursive Implementierung. 

Wir arbeiten dafür mit einem weiteren Array `C`, das mit Null initialisiert wird. Dann werden wir die einzelnen Cluster suchen und durchnummerieren. Am Ende soll in `C` auf jedem Gitterplatz, der zu einem Cluster gehört die Nummer des Clusters stehen, oder Null, falls der Platz zu keinem Cluster gehört.

Wir schreiben dafür eine Funktion `find_cluster!`, die überprüft ob ein Gitterplatz $i,j$ besetzt ist und ob er bereits zu einem Cluster gehört (also ob `C[i,j]>0`). Falls der Platz besetzt ist, wird die Rekursion beendet. Falls der Gitterplatz nicht besetzt ist, wird er dem aktuellen Cluster hinzugefügt und `find_cluster!` wird für die benachbarten Gitterplätze aufgerufen.

Die Funktion soll außerdem direkt die Clustergröße bestimmen, daher zählen wir mit wie viele Gitterplätze hinzugefügt werden und geben die Zahl zurück.

In [ ]:
function find_cluster!(clusterIdx, i,j,C,G,step=1)

end

Test der Funktion:

In [ ]:
C=zeros(Int, 10,10)

find_cluster!(17,1,1,C,G)

In [ ]:
C

Um den größten Cluster zu finden, starten wir `find_cluster!` auf jedem Gitterplatz um alle Cluster zu identifizieren. In der Schleife merken wir uns gleich welche die maximale Clustergröße ist.

In [ ]:
function find_largest_cluster(G)

end

maxSize, C = find_largest_cluster(G)

In [ ]:
imshow(C)
axis("off");

Jetzt berechnen wir für eine Reihe von Besetzungswahrscheinlichkeiten $p\in[0,1]$ die maximalen Clustergrößen und tragen sie im Plot auf:

In [ ]:
Random.seed!(42)

N=100
ps=range(0,1,step=0.05)
mean_cluster_sizes=[]

for p in ps
    tmp_sizes=[]
    for j in 1:100
        G=get_configuration(N,p)
        s, C = find_largest_cluster(G)
        push!(tmp_sizes, s)
    end
    push!(mean_cluster_sizes, mean(tmp_sizes))
end

plot(ps, mean_cluster_sizes/N^2)
xlabel("Besetzungswahrscheinlichkeit")
ylabel("Normierte Clustergröße");

# Verkehrssimulation

Wir wollen den Verkehrsfluss auf einer einspurigen Straße simulieren und dabei die Abhängigkeit von der Verkehrsdichte $\rho$ untersuchen.

Wir modellieren das Problem wie folgt: Die Straße besteht aus $L$ diskreten Positionen auf einem Ring (periodische Randbedingungen). Jede Position kann leer oder mit einem Fahrzeug besetzt sein. Die Fahrzeuge bewegen sich in diskreten Zeitschritten nach den folgenden Regeln:

1. Falls die nächsten zwei Gitterplätze vor einem Fahrzeug frei sind, rückt es um einen Platz vor.
2. Falls der nächste Gitterplatz vor einem Fahrzeug frei ist, der übernächste aber besetzt, rückt das Fahrzeug mit einer Wahrscheinlichkeit von $p=0.5$ vor (Bremsen bzw. verzögertes Anfahren).
3. Falls der nächste Gitterplatz vor einem Fahrzeug besetzt ist, rückt es nicht vor.

In [ ]:
function get_initial_config(rho, L)

end

L=100
rho=0.3
config = get_initial_config(rho, L)

In [ ]:
sum(config)

Jetzt implementieren wir eine Funktion, die für eine gegebenen Fahrzeugkonfiguration einen Zeitschritt entsprechend der Regeln durchführt. Gleichzeitig ermittelt die Funktion die Mittlere Geschwindigkeit, also die Zahl der vorgerückten Fahrzeuge geteilt durch die Gesamtzahl an Fahrzeugen.

In [ ]:
idx(n,L)=1+(n-1)%L

function step(config)

end

Num können wir uns einen "Film" des Verkehrsflusses ansehen:

In [ ]:
using Printf

function show(rho, L, steps=100)
    pygui(true)
    config = get_initial_config(rho, L)

    axis("off")
    for t in 1:steps
        config, v = step(config)
        imshow(reshape(config,(1,L)))
        title(@sprintf("Geschwindigkeit: %.2f", v))
        sleep(0.2)
    end
    pygui(false);
    return nothing
end

show(0.4,100, 100)

Bestimme die mittlere Fließgeschwindigkeit in Abhängigkeit von der Verkehrsdichte $\rho$:

In [ ]:
Random.seed!(23)

rhos=range(0.05,0.95,step=0.05)

L=1000

v_mean=[]

for rho in rhos
    config = get_initial_config(rho, L)
    v_sum = 0
    for t in 1:10000
        config, v = step(config)
        v_sum += v
    end
    push!(v_mean, v_sum/10000)
end

plot(rhos, v_mean);
xlabel("Verkehrsdichte")
ylabel("Fließgeschwindigkeit");

Typische Situationen: Freie Fahrt und Stau

In [ ]:
show(0.15,100);

In [ ]:
show(0.8,100);

[Hier](https://www.youtube.com/watch?v=7wm-pZp_mi0]) findet ihr ein Video eines ähnlichen Experiments mit echten Fahrzeugen auf einer einspurigen Fahrbahn, das zeigt wie ein Stau aus dem Nichts entsteht.

# Problemlösungsalgorithmus für numerische Experimente

Numerische Experimente werden schnell komplex. Daher sollte man beim Lösen des Problems schrittweise vorgehen um den Überblick zu behalten und Fehler frühzeitig zu identifizieren:

1. Analysieren des Problems: Gleichungen verstehen und numerische Aufgaben identifizieren.
2. Zerlegen des Problems in Teilprobleme.
3. Lösungen der Teilprobleme implementieren (separate Funktionen).
4. Lösungen mit einfachen Beispielen testen.
5. Daten produzieren.
6. Daten analysieren. Das Ergebnis auf Plausibilität prüfen.